<a href="https://colab.research.google.com/github/pranaysingh97-bit/Harry-Potter-Agent/blob/main/Harrypotter_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the Google GenAI SDK and a powerful agent framework
!pip install -q google-genai crewai crewai-tools

In [ ]:
import os
from google.colab import userdata

# Get your key from Google AI Studio and save it as a Colab Secret named 'GEMINI_API_KEY'
if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

# The CrewAI LLM wrapper for Gemini
from crewai import LLM
from crewai_tools import SerperDevTool # Tool for the agent to look up information

# Initialize the Gemini model for your agent
# Gemini 2.5 Flash is fast and capable for this creative task
my_llm = LLM(
    model='gemini/gemini-2.5-flash',
    api_key=os.environ["GEMINI_API_KEY"],
    temperature=0.7 # Set a creative temperature
)

In [ ]:
persona_backstory = (
    "You are J.K. Rowling, the legendary author of the 'Harry Potter' series. "
    "You now write insightful, engaging LinkedIn posts to teach modern career "
    "and life lessons, always using a creative analogy from the Harry Potter books. "
    "Your tone is wise, inspiring, and slightly mystical, maintaining the spirit "
    "of an author speaking to her adult readers. You must always reference at "
    "least one key character (like Dumbledore, Snape, or Hermione) in your post."
)

In [ ]:
# Create a tool for real-time information lookup
# You'll need to set a SERPER_API_KEY in your Colab Secrets as well
search_tool = SerperDevTool(api_key=userdata.get("SERPER_API_KEY"))

# Note: The ReAct pattern is 'Thought -> Action (Tool) -> Observation -> Thought -> Final Answer'
# The agent will use this tool when it needs external knowledge.

In [ ]:
from crewai import Agent, Task, Crew, Process

# 🧙 The J.K. Rowling Agent
rowling_agent = Agent(
    role='LinkedIn Post Creator & Analogy Specialist',
    goal='Create a highly engaging, original LinkedIn post on a user-provided topic, '
         'grounded in the persona of J.K. Rowling, and using an insightful Harry Potter analogy.',
    backstory=persona_backstory,
    tools=[search_tool], # Gives the agent the ability to 'Act' (A) based on 'Reasoning' (R)
    llm=my_llm,
    verbose=True, # Set to True to see the internal ReAct steps (Thought, Action, Observation)
    # The CrewAI framework provides memory by default for the entire task context.
)

# 📝 The Specific Task
def create_linkedin_task(topic):
    return Task(
        description=(
            f"Write a single, creative LinkedIn post (4-5 paragraphs, maximum 200 words) "
            f"about the topic: '{topic}'. The post MUST: "
            f"1. Start with an engaging, hook-like opening sentence."
            f"2. Incorporate an analogy from the 'Harry Potter' universe. "
            f"3. Name at least one character (e.g., Albus Dumbledore, Severus Snape, Hagrid) to illustrate the lesson. "
            f"4. End with a question to drive engagement (e.g., 'What's your biggest life lesson from Hogwarts?'). "
            f"5. Include relevant LinkedIn hashtags (e.g., #Leadership #Career #HarryPotter #CreativeThinking)."
        ),
        agent=rowling_agent,
        expected_output="A final, polished LinkedIn post, ready to copy-paste.",
    )

In [ ]:
# --- User Input ---
topic = input("Enter the topic for J.K. Rowling's LinkedIn post (e.g., 'Overcoming self-doubt in a new career'): ")

# --- Setup the Crew ---
crew = Crew(
    agents=[rowling_agent],
    tasks=[create_linkedin_task(topic)],
    verbose=True, # Verbose level 2 shows agent and task execution details
    process=Process.sequential # Since it's a single agent, the process is sequential
)

# --- Execute the Agent ---
print("\n" + "="*50)
print(f"🪄 Generating J.K. Rowling's Post on: {topic}")
print("="*50)

# The result of this execution is your final, creative post
result = crew.kickoff()

print("\n" + "="*50)
print("✨ FINAL GENERATED LINKEDIN POST ✨")
print("="*50)
print(result)